In [1]:
import numpy as np
import pandas as pd
import BasicModel as bm
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests

In [162]:
#获取三个市场的基本数据
fc = bm.basicmodel()
fc.ret(start='2011-07-01',end='2013-07-01')
fc.AR_GARCH()

Iteration:      1,   Func. Count:      6,   Neg. LLF: -2618.905264873367
Inequality constraints incompatible    (Exit mode 4)
            Current function value: -2618.9052652253695
            Iterations: 1
            Function evaluations: 6
            Gradient evaluations: 1
Iteration:      1,   Func. Count:      7,   Neg. LLF: 5628214106797739.0
Iteration:      2,   Func. Count:     22,   Neg. LLF: 3322488190297517.0
Iteration:      3,   Func. Count:     34,   Neg. LLF: 5.423050007298206e+20
Iteration:      4,   Func. Count:     49,   Neg. LLF: 7.970336144262167e+20
Iteration:      5,   Func. Count:     65,   Neg. LLF: 28837125183904.984
Iteration:      6,   Func. Count:     79,   Neg. LLF: 6.261732885524452e+20
Iteration:      7,   Func. Count:     95,   Neg. LLF: 3.3017432761404326e+23
Iteration:      8,   Func. Count:    108,   Neg. LLF: -2427.407010685405
Optimization terminated successfully    (Exit mode 0)
            Current function value: -2427.4070107504126
            I

c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 8.224e-07. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:765: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
c:\Users\86177\anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 3.808e-06. Parameter
estimation work better when this value is between 1 and 1000. T

In [133]:
model = VAR(fc.std_index[['CNY','CNH']])
for i in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)

Lag Order = 1
AIC :  -5.617121374857527
BIC :  -5.523667991810326
Lag Order = 2
AIC :  -7.598347175439663
BIC :  -7.442084286195026
Lag Order = 3
AIC :  -7.719221086361082
BIC :  -7.499737679878351
Lag Order = 4
AIC :  -8.02499556217386
BIC :  -7.741876084274825
Lag Order = 5
AIC :  -8.092700109518754
BIC :  -7.74552439287271
Lag Order = 6
AIC :  -8.071042715312169
BIC :  -7.659385908041073
Lag Order = 7
AIC :  -8.060397399527496
BIC :  -7.583829892398198
Lag Order = 8
AIC :  -8.183055695501949
BIC :  -7.641143047671502
Lag Order = 9
AIC :  -8.185484829943132
BIC :  -7.577787693222256
Lag Order = 10
AIC :  -8.19927799150149
BIC :  -7.525352033100261
Lag Order = 11
AIC :  -8.175993726088969
BIC :  -7.435389549807776
Lag Order = 12
AIC :  -8.152590721649368
BIC :  -7.34485378747714
Lag Order = 13
AIC :  -8.132130529834027
BIC :  -7.2568010719146425
Lag Order = 14
AIC :  -8.124901751232125
BIC :  -7.1815146941587695
Lag Order = 15
AIC :  -8.081727727297512
BIC :  -7.069812600693616


c:\Users\86177\anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


In [134]:
def grangers_causation_matrix(data,maxlag,test='ssr_chi2test', verbose=False):    
    #maxlag可以是最大值，也可以是所有需要测试的阶数的一个迭代器
    if type(maxlag) == int:
        alltestlag = [i+1 for i in range(maxlag)]
    else:
        alltestlag = maxlag
    #生成一个pvalue矩阵
    variables = data.columns
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i][0][test][1],4) for i in alltestlag]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_cause' for var in variables]
    df.index = [var + '_reason' for var in variables]
    return df

In [164]:
grangers_causation_matrix(fc.std_index[:150],maxlag=7)

,CNY_cause,CNH_cause,NDF_cause
CNY_reason,1.0000,0.0653,0.0174
CNH_reason,0.3261,1.0000,0.4111
NDF_reason,0.2873,0.0424,1.0000
